In [12]:
import sys
import math
import pandas as pd
import numpy as np

In [13]:
merchant_category_path = '/Users/xtx/Affirm_QR_Take_Home/merchant_category_mapping.csv'
merchant_name_mapping_path = '/Users/xtx/Affirm_QR_Take_Home/merchant_name_mapping.csv'
perf_dataset_path = '/Users/xtx/Affirm_QR_Take_Home/performance_dataset.csv'

In [14]:
merchant_category = pd.read_csv(merchant_category_path)
merchant_name_mapping = pd.read_csv(merchant_name_mapping_path)
perf_dataset = pd.read_csv(perf_dataset_path)

df_merge=perf_dataset.merge(merchant_name_mapping, on='merchant_id', how='left')
df_merge=perf_dataset.merge(merchant_category, on='merchant_id', how='left')

In [15]:
df_merge['pred_diff'] = df_merge['predicted_repayment_pct']-df_merge['actual_repayment_pct']
df_merge['dollar_pred_diff'] = df_merge['pred_diff'] * df_merge['num_trxn'] * df_merge['avg_loan_amt']
df_merge['$_loan_total']= df_merge['num_trxn'] * df_merge['avg_loan_amt']
df_merge['int_income'] = df_merge['$_loan_total'] * df_merge['avg_apr'] * df_merge['avg_term']/12
df_merge['bad_loss'] = (1-df_merge['actual_repayment_pct'])*df_merge['$_loan_total']
df_merge['pnl'] = df_merge['int_income'] - df_merge['bad_loss']
print(df_merge.head())

   merchant_id  actual_repayment_pct  predicted_repayment_pct  num_trxn  \
0        78986              0.927466                 0.948425      6331   
1        54871              0.983798                 0.978641      3293   
2        86469              0.936782                 0.965042      2670   
3        42719              0.961743                 0.945634       570   
4        40242              0.867273                 0.902408       317   

   avg_auth_amt  avg_loan_amt  avg_fico  avg_term   avg_apr          category  \
0    491.650278    473.727737       643         8  0.251198    WOMENS_FASHION   
1    862.996710    862.555109       715         6  0.000000  HOME_FURNISHINGS   
2    636.689936    620.229734       664         8  0.214657               NaN   
3    719.714054    692.877934       655        10  0.257577             OTHER   
4    380.978427    374.832065       626         9  0.160714           JEWELRY   

                                       subcategory  pred_diff 

In [16]:
print("Total Over prediction : $", df_merge['dollar_pred_diff'].sum())
print("% of over prediction : ", (df_merge['dollar_pred_diff'].sum()/(df_merge['$_loan_total'].sum())*100))

Total Over prediction : $ 155270.61790686258
% of over prediction :  1.3986898510558483


In [17]:
df_overpredict = df_merge.query('predicted_repayment_pct > actual_repayment_pct')
df_underpredict = df_merge.query('predicted_repayment_pct < actual_repayment_pct')
print("DESCRIPTIVE STATISTICS : OVER Prediction\n\n", df_overpredict[['predicted_repayment_pct', 'actual_repayment_pct', 'pred_diff']].describe())
print("\nDESCRIPTIVE STATISTICS : UNDER Prediction \n\n",df_underpredict[['predicted_repayment_pct', 'actual_repayment_pct', 'pred_diff']].describe())

DESCRIPTIVE STATISTICS : OVER Prediction

        predicted_repayment_pct  actual_repayment_pct  pred_diff
count                55.000000             55.000000  55.000000
mean                  0.933767              0.797435   0.136332
std                   0.046910              0.221310   0.202763
min                   0.707895              0.000000   0.002636
25%                   0.928326              0.815830   0.033732
50%                   0.947439              0.867273   0.073108
75%                   0.959642              0.922428   0.127173
max                   0.974091              0.969390   0.950731

DESCRIPTIVE STATISTICS : UNDER Prediction 

        predicted_repayment_pct  actual_repayment_pct  pred_diff
count                88.000000             88.000000  88.000000
mean                  0.942248              0.989509  -0.047261
std                   0.040082              0.023499   0.041997
min                   0.715887              0.882083  -0.284113
25%            

In [18]:
df_merge=df_merge.sort_values(by=['pnl'], ascending=False)
print("Top 10 Profitable Merchants :\n\n")
print(df_merge[:10])

Top 10 Profitable Merchants :


    merchant_id  actual_repayment_pct  predicted_repayment_pct  num_trxn  \
0         78986              0.927466                 0.948425      6331   
2         86469              0.936782                 0.965042      2670   
3         42719              0.961743                 0.945634       570   
10        53810              0.948553                 0.963008       164   
24        13615              0.965838                 0.972367        59   
14        37889              0.944777                 0.937538       141   
15        47974              0.952664                 0.946363       107   
11        12368              0.893519                 0.947609       161   
20        36413              0.973708                 0.958271        77   
17        67386              0.937309                 0.922502        89   

    avg_auth_amt  avg_loan_amt  avg_fico  avg_term   avg_apr  \
0     491.650278    473.727737       643         8  0.251198   
2  

In [19]:
print("Top 10 Loss making Merchants :\n\n")
print(df_merge[-10:])

Top 10 Loss making Merchants :


     merchant_id  actual_repayment_pct  predicted_repayment_pct  num_trxn  \
115        81395              0.500000                 0.953412         2   
7          27822              0.855525                 0.916094       221   
33         26696              0.969390                 0.974091        35   
4          40242              0.867273                 0.902408       317   
5          11801              0.822480                 0.900832       309   
35         83349              0.739957                 0.929764        33   
27         76187              0.941740                 0.973775        49   
34         40512              0.714083                 0.926887        33   
19         37559              0.974491                 0.964564        88   
1          54871              0.983798                 0.978641      3293   

     avg_auth_amt  avg_loan_amt  avg_fico  avg_term   avg_apr  \
115   1232.500000   1207.916666       674        11  0